In [4]:
using Flux
include("preprocessing.jl")

reconstruct_midi_file (generic function with 1 method)

In [13]:
function prepare_data(input_dir::String)
    inputs = []
    outputs = []

    for csv in readdir(input_dir)
        df = CSV.read(joinpath(input_dir, csv), DataFrame)
        push!(inputs, df[:, 1:end-1])
        push!(outputs, df[:, end])
    end
    
    (inputs, outputs)
end

inputs, outputs = prepare_data("assets/anomalous")

size(inputs)
size(outputs)

(10260,)

In [18]:
println(size(inputs))
println(size(outputs))

Any[

940×4 DataFrame
 Row │ note   velocity  position  duration 
     │ Int64  Int64     Int64     Int64    
─────┼─────────────────────────────────────
   1 │    60        45         0       935
   2 │    33        17         0       935
   3 │    69        41       440       495
   4 │    36        25       440       495
   5 │    40        17       605       330
   6 │    45        33       715       220
   7 │    55        41       825       770
   8 │    45        25       990       605
   9 │    64        41       990       605
  10 │    67        49      1100       495
  11 │    52        25      1100       495
  12 │    70        49      1210       385
  13 │    61        16      1575       436
  14 │    57        45      1430       165
  15 │    38        25      1595       825
  16 │    57        49      1595       660
  17 │    65        45      1760       660
  18 │    69        41      1870       550
  19 │    50        41      1980       440
  20 │    53        25      2090   

, 940×4 DataFrame
 Row │ note   velocity  position  duration 
     │ Int64  Int64     Int64     Int64    
─────┼─────────────────────────────────────
   1 │    60        45         0       935
   2 │    33         0       315       935
   3 │    58        19       440      1062
   4 │    36        25       440       495
   5 │    40        17       605       330
   6 │    45        33       715       220
   7 │    55        41       825       770
   8 │    45        25       990       605
   9 │    64        41       990       605
  10 │    67        49      1100       495
  11 │    52        25      1100       495
  12 │    70        49      1210       385
  13 │    61        41      1320       275
  14 │    57        45      1430       165
  15 │    38        25      1595       825
  16 │    57        49      1595       660
  17 │    65        45      1760       660
  18 │    69        41      1870       550
  19 │    50        41      1980       440
  20 │    59         0      2361 

, 940×4 DataFrame
 Row │ note   velocity  position  duration 
     │ Int64  Int64     Int64     Int64    
─────┼─────────────────────────────────────
   1 │    60        45         0       935
   2 │    33        41         0      1532
   3 │    69        41       440       495
   4 │    36        25       440       495
   5 │    40        17       605       964
   6 │    45        33       715       220
   7 │    55        41       825       770
   8 │    45        25       990       605
   9 │    64        41       990       605
  10 │    67        49      1100       495
  11 │    52        25      1100       495
  12 │    70        49      1210       385
  13 │    61        41      1320       275
  14 │    71        84      1039       563
  15 │    38        25      1595       825
  16 │    57        49      1595       660
  17 │    65        45      1760       660
  18 │    69        41      1870       550
  19 │    50        41      1980       440
  20 │    53        25      2090 

, 940×4 DataFrame
 Row │ note   velocity  position  duration 
     │ Int64  Int64     Int64     Int64    
─────┼─────────────────────────────────────
   1 │    60        45         0       935
   2 │    33        17         0       935
   3 │    69        18       293       726
   4 │    36        25       440       495
   5 │    40        17       605       330
   6 │    45        33       715       220
   7 │    55        41       825       770
   8 │    45        46       990       525
   9 │    64        41       990       605
  10 │    67        49      1100       495
  11 │    52        25      1100       495
  12 │    70        49      1210       385
  13 │    61        41      1320       275
  14 │    57        45      1430       165
  15 │    38        25      1595       825
  16 │    57        49      1595       660
  17 │    65        16      1921       890
  18 │    69        41      1870       550
  19 │    50        41      1980       440
  20 │    53        25      2090 

...

, 940×4 DataFrame
 Row │ note   velocity  position  duration 
     │ Int64  Int64     Int64     Int64    
─────┼─────────────────────────────────────
   1 │    54        20       230      1767
   2 │    33        17         0       935
   3 │    69        41       440       495
   4 │    42        54        38       495
   5 │    31         0      1034       330
   6 │    45        33       715       220
   7 │    55        80       825      1513
   8 │    45        25       990       605
   9 │    64        41       990       605
  10 │    71        49      1187       495
  11 │    52        25      1100       495
  12 │    70        74      1210      1307
  13 │    49        41      1320       275
  14 │    57         5      1430       940
  15 │    36        25      1595      1168
  16 │    57        49      1342       660
  17 │    59        15      2190       660
  18 │    69        41      1870       837
  19 │    50        41      1980       440
  20 │    53        25      2090 

, 

940×4 DataFrame
 Row │ note   velocity  position  duration 
     │ Int64  Int64     Int64     Int64    
─────┼─────────────────────────────────────
   1 │    65        45         0       935
   2 │    33        17       335      1032
   3 │    69        41       440       545
   4 │    36        56       516       495
   5 │    32        17       605       330
   6 │    45        33       715       220
   7 │    41        15      1029      1359
   8 │    45        25       990       605
   9 │    63        71       990       605
  10 │    69        85      1100       495
  11 │    52         0      1318       495
  12 │    70        49      1210       385
  13 │    63        62      1320       275
  14 │    57        45      1430       165
  15 │    38        25      1595       825
  16 │    45        26      1450       660
  17 │    65        45      1760       660
  18 │    70        41      1712       861
  19 │    50        41      1980       440
  20 │    53        25      2090   

Excessive output truncated after 554186 bytes.

, 

In [ ]:
function train_model(filename)
    input_size = 4
    output_size = 1
    hidden_size = 128

    inputs, outputs = load_data(filename)

    model = Chain(
        LSTM(input_size, hidden_size),
        LSTM(hidden_size, hidden_size),
        Dense(hidden_size, output_size)
    )

    loss(x, y) = Flux.mse(model(x), y)
    opt = ADAM()

    Flux.train!(loss, params(model), [(inputs, outputs)], opt)

    return model
end
